In [1]:
import numpy as np
import pandas as pd

import time
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


# 1. Data Preparation

3 datasets:

S&P 500 daily adjusted price

Mean-variance weights table

Min-variance weights table

In [10]:
# data_adj = pd.read_csv("final_ratios_oct.csv")
# weight_meanv = pd.read_excel('mean_weighted_rf.xlsx')
# weight_minv = pd.read_excel('minimum_weighted_rf.xlsx')
# weight_equal=pd.read_excel('equally_weighted_rf.xlsx')
data_adj = pd.read_csv("./data_processor_update/outputs/final_ratios.csv")
weight_meanv = pd.read_excel('mean_weighted.xlsx')
weight_minv = pd.read_excel('minimum_weighted.xlsx')
weight_equal = pd.read_excel('equally_weighted.xlsx')

In [11]:
data_adj

,Unnamed: 0,date,gvkey,tic,gsector,adj_close_q,y_return,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,acc_rec_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,0,2000-01-31,126554,A,35,39.880585,-2.869590e-07,0.30,9.924779,0.000,2.410256,1.381229,0.682148,4.529412,0.368791,0.000000,330.937500,16.162480,6.668914
1,1,2000-01-31,126554,A,35,39.880573,0.000000e+00,0.30,9.924779,0.000,2.410256,1.381229,0.682148,4.419377,0.368791,0.000000,330.937500,16.162480,6.668914
2,2,2000-07-31,126554,A,35,24.553490,-3.884069e-07,1.01,10.820834,0.000,2.129084,1.143426,0.280080,2.696816,0.373706,0.000000,127.343750,7.852132,3.765884
3,3,2000-07-31,126554,A,35,24.553480,1.279559e-01,1.01,10.820834,0.000,2.129084,1.143426,0.280080,2.927550,0.373706,0.000000,127.343750,7.852132,3.765884
4,4,2000-10-31,126554,A,35,27.905100,3.417562e-07,1.68,11.597529,0.000,2.050399,1.159173,0.361131,3.109496,0.375074,0.000000,70.170455,6.968765,3.993307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89701,89701,2023-06-30,13721,ZTS,35,169.228973,0.000000e+00,2.64,10.037960,0.375,3.495195,1.717920,0.970605,4.726929,0.663903,1.975330,118.765517,36.380527,17.170727
89702,89702,2024-09-30,13721,ZTS,35,194.330627,0.000000e+00,4.18,11.591186,0.432,3.690952,1.836075,1.007051,4.796598,0.635439,1.743026,129.390728,36.913740,16.855911
89703,89703,2024-09-30,13721,ZTS,35,194.330627,-1.792589e-01,4.18,11.591186,0.432,3.690952,1.836075,1.007051,4.921332,0.635439,1.743026,129.390728,36.913740,16.855911
89704,89704,2024-12-31,13721,ZTS,35,162.438919,0.000000e+00,5.47,10.636092,0.432,1.754396,0.968054,0.582356,5.427052,0.664958,1.984696,126.302326,31.536343,15.318597


In [12]:
weight_meanv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2018-03-01,1045,0.00625,0.034466
1,1,2018-03-01,1075,0.00625,0.036502
2,2,2018-03-01,1161,0.00625,0.032784
3,3,2018-03-01,1230,0.00625,0.035069
4,4,2018-03-01,2220,0.00625,0.025938


In [13]:
weight_minv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2018-03-01,1045,0.00625,0.034466
1,1,2018-03-01,1075,0.00625,0.036502
2,2,2018-03-01,1161,0.00625,0.032784
3,3,2018-03-01,1230,0.00625,0.035069
4,4,2018-03-01,2220,0.00625,0.025938


In [14]:
weight_equal.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2018-03-01,1045,0.00625,0.034466
1,1,2018-03-01,1075,0.00625,0.036502
2,2,2018-03-01,1161,0.00625,0.032784
3,3,2018-03-01,1230,0.00625,0.035069
4,4,2018-03-01,2220,0.00625,0.025938


In [15]:
#the length of the 3 tables should be the same
print(len(weight_meanv),
len(weight_minv),
len(weight_equal))


3784 3784 3784


## 1.1 Get stock name, trade date, daily date

Add 20170901 as tradedate_full


In [16]:
#get unique stock name from table
#stock names are the same between mean variance and minimum variance
#only the weights are different
stocks_name = list(set(weight_meanv['gvkey']))
stocks_name.sort()

In [17]:
len(stocks_name)

607

In [18]:
#get all daily date
#convert to list from pandas.core.series.Series
#use set to get unique values
daily = list(set(data_adj['date']))
#sort it
daily.sort()
print(type(data_adj['date']),type(daily))

<class 'pandas.core.series.Series'> <class 'list'>


In [19]:
len(daily)


245

In [20]:
#get unique trade date from table
tradedate = list(set(weight_meanv['trade_date']))
tradedate.sort()


In [21]:
tradedate

['2018-03-01',
 '2018-06-01',
 '2019-03-01',
 '2020-06-01',
 '2020-09-01',
 '2020-12-01',
 '2021-03-01',
 '2021-06-01',
 '2021-09-01',
 '2021-12-01',
 '2022-03-01',
 '2022-06-01',
 '2022-09-01',
 '2022-12-01',
 '2023-03-01',
 '2023-06-01',
 '2023-09-01',
 '2023-12-01',
 '2024-03-01']

In [58]:
tradedate_full = [x for x in tradedate if x > '2018-01-01']

In [59]:
tradedate_full

['2018-03-01',
 '2018-06-01',
 '2019-03-01',
 '2020-06-01',
 '2020-09-01',
 '2020-12-01',
 '2021-03-01',
 '2021-06-01',
 '2021-09-01',
 '2021-12-01',
 '2022-03-01',
 '2022-06-01',
 '2022-09-01',
 '2022-12-01',
 '2023-03-01',
 '2023-06-01',
 '2023-09-01',
 '2023-12-01',
 '2024-03-01']

## 1.2 Get the daily price matrix based on stock_name

In [60]:
#30mins running time
##################match current all stocks to daily adjusted price table#######################
# backtest daily price matrix:
# covert original table to matrix
# rows = all stocks: 971, columns = all dates:7155 days
#
start_time = time.time()

balance_daily = pd.DataFrame(index = stocks_name, columns = daily)
# balance_daily = balance_daily.fillna(np.nan)
balance_daily = balance_daily.astype(float)

# Convert both stocks_name and gvkey to strings for consistency
data_adj.loc[:, "gvkey"] = data_adj["gvkey"].astype(str)  
stocks_name = [str(s) for s in stocks_name]
# Find stocks in stocks_name that are NOT in data_adj['gvkey']
missing_stocks = set(stocks_name) - set(data_adj["gvkey"].unique())

if missing_stocks:
    print(f"Warning: These stocks are missing from data_adj: {missing_stocks}")



for i in stocks_name:
    # ind = np.where(data_adj['gvkey'] == i)[0]
    # tmp = data_adj.loc[ind, :]
    tmp = data_adj[data_adj["gvkey"] == i]  # No need for np.where()
    tmp.index = range(tmp.shape[0])
    for j in daily:
        # ind2 = np.where(tmp['date'] == j)[0]
        match = tmp[tmp["date"] == j]
        # if(len(ind2) != 0):
        if not match.empty:
            # balance_daily.loc[i,j] = tmp.loc[ind2[0], 'adj_close_q']
            balance_daily.loc[i, j] = match.iloc[0]["adj_close_q"]

elapsed_time = time.time() - start_time
print(elapsed_time)
# writer2 = pd.ExcelWriter('balance_daily.xlsx')
# writer2 = pd.ExcelWriter('balance_daily_user8.xlsx')

# balance_daily.to_excel(writer2,'Sheet1')
# writer2.save()
with pd.ExcelWriter("balance_daily_user8.xlsx", engine="openpyxl") as writer2:
    balance_daily.to_excel(writer2, sheet_name="Sheet1")

###############################################

12.95959186553955


In [61]:
balance_daily = pd.read_excel('balance_daily_user8.xlsx', index_col=0)


In [62]:
balance_daily

,1996-01-31,1996-02-29,1996-04-30,1996-05-31,1996-07-31,1996-09-30,1996-10-31,1996-12-31,1997-01-31,1997-02-28,...,2023-11-30,2024-01-31,2024-02-29,2024-04-30,2024-05-31,2024-07-31,2024-09-30,2024-10-31,2024-12-31,2025-01-31
1045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.240000,NaN,17.430000,NaN
1075,NaN,NaN,NaN,NaN,NaN,8.758753,NaN,9.470697,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,86.763947,NaN,83.897537,NaN
1078,NaN,NaN,NaN,NaN,NaN,5.481436,NaN,5.675969,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,112.882820,NaN,112.519531,NaN
1161,NaN,NaN,NaN,NaN,NaN,7.375000,NaN,12.875000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,164.080002,NaN,120.790001,NaN
1209,NaN,NaN,NaN,NaN,NaN,14.152513,NaN,16.861080,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,294.163818,NaN,288.269989,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,94.358055,NaN,74.269997,NaN
312009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22.850000,NaN,14.190000,NaN
316056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,145.740005,NaN,130.679993,NaN
322985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Function: calculate the rebalance matrix:

1. shares table
2. cost table
3. cash table
4. portfolio table
5. portfolio_cost table
6. portfolio_return table
7. portfolio_cumsum table

In [63]:
# set initial capital to 1 million and transaction percet to 0.1%

capital = 1000000
transaction_percent = 1/1000

In [64]:
weight_meanv.shape

(3784, 5)

In [102]:
#construct an empty matrix
#rows are stocks: 971 unique stocks
#columns are trade dates
#name the index as stock names

def cal_portfolio(stocks_name,tradedate, weight_table, capital, transaction_percent):
    tradedate_full = tradedate #cuz arent they the same here?

    balance_weight = pd.DataFrame(index = stocks_name, columns = tradedate)
    # balance_weight = balance_weight.fillna(0)
    balance_weight.astype(float).fillna(0)
    
    #loop through the weights table to fill the matrix
    #need weight tabele
    #0. weights table
    for i in range(weight_table.shape[0]):
        tics = weight_table.loc[i, 'gvkey']
        date = weight_table.loc[i, 'trade_date']
        w = weight_table.loc[i, 'weights']
        balance_weight.loc[tics, date] = w
        
    #1. shares table
    balance_share = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_share['initial'] = 0
    # balance_share = balance_share.fillna(0)
    balance_share = balance_share.astype(float).fillna(0)
    #2. costs table
    balance_cost = pd.DataFrame(index = stocks_name, columns = tradedate)
    # balance_cost = balance_cost.fillna(0)
    balance_cost = balance_cost.astype(float).fillna(0)
    #3. cash table
    balance_cash = pd.DataFrame(index = stocks_name, columns = tradedate)
    # balance_cash = balance_cash.fillna(0)
    balance_cash = balance_cash.astype(float).fillna(0)

    #4. portfolio without transaction costs table 
    portfolio = pd.DataFrame(index = range(1), columns = tradedate_full)
    portfolio.loc[0,tradedate[0]] = capital
    
    #missing dates
    balance_daily.columns = pd.to_datetime(balance_daily.columns)
    tradedate_full = pd.to_datetime(tradedate_full)
    missing_dates = set(tradedate) - set(balance_daily.columns)
    print("Missing Dates:", missing_dates)
    tradedate_full = [d for d in tradedate_full if d in balance_daily.columns]
    balance_price_full = balance_daily[tradedate_full]



    # calculate portfolio value without cost
    balance_price_full = []
    
    balance_daily.columns = pd.to_datetime(balance_daily.columns)
    tradedate = pd.to_datetime(tradedate)
    
    # Remove dates that are not in balance_daily
    tradedate = [d for d in tradedate if d in balance_daily.columns]
    
    # Now access balance_daily safely
    balance_price = balance_daily[tradedate]
    
    balance_price_full = balance_daily[tradedate_full]
    # balance_price = balance_daily[tradedate]
    balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)
    for j in range(len(tradedate) - 1):
        i = tradedate[j]
        #current capital
        cap = portfolio.loc[0, i]
        #
        balance_cash[i] = cap * balance_weight[i]
        balance_share[i] = list(np.array(balance_cash[i])
                                      / np.array(balance_price[i]))
        balance_share[i] = balance_share[i].fillna(0)
        balance_share[i] = balance_share[i].astype(int)
        #next quarter price * current share to calculate next quarter total account value
        a = balance_price_full[tradedate_full[j+1]].fillna(0)
        portfolio.loc[0, tradedate_full[j+1]] = np.dot(balance_share[i], a)

    # balance_cost = abs(np.diff(balance_share, axis = 1))
    balance_cost = balance_share.diff(axis=1).fillna(0).abs()
    balance_cost = balance_cost.reindex(columns=balance_price.columns, fill_value=0)
    print("balance_cost shape:", balance_cost.shape)
    print("balance_price shape:", balance_price.shape)


    balance_cost = balance_cost * balance_price
    balance_cost = np.sum(balance_cost, axis = 0) * transaction_percent

    if balance_cost.empty:
        raise ValueError("🚨 Error: `balance_cost` is empty. Check its computation earlier.")
    print("portfolio_cost shape:", portfolio_cost.shape)
    print("tradedate_full length:", len(tradedate_full))
    if portfolio_cost.shape[0] != len(tradedate_full):
        raise ValueError(f"Shape mismatch: portfolio_cost has {portfolio_cost.shape[0]} elements, but tradedate_full has {len(tradedate_full)}.")

    #5. portfolio with transaction cost table
    #calcualte portolio value with transaction cost
    balance_cost.index = tradedate_full
    # portfolio_cost = []
    # portfolio_cost = portfolio.loc[0,] - balance_cost
    portfolio_cost = portfolio.loc[0, :].copy() - balance_cost

    portfolio_cost.index = tradedate_full
    portfolio_cost_value = portfolio_cost.copy()
    
    #6. portfolio return P&L table
    portfolio_cumsum = portfolio_cost_value.copy()
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp = portfolio_cumsum.shift(1)
    tmp[0] = 0
    tmp.index = tradedate_full

    portfolio_cumsum = portfolio_cost_value- tmp
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp.index = range(len(tmp))
    for i in range(len(portfolio_cumsum)):
        if(tmp[i] != 0):
            portfolio_cumsum[i] = portfolio_cumsum[i]/tmp[i]

    portfolio_cumsum.index = range(len(portfolio_cumsum))
    portfolio_cumsum[0] = 0
    portfolio_return=portfolio_cumsum.copy()



    # 7. portfolio accumulative return table
    initial = 0
    for i in range(portfolio_cumsum.shape[0]):
        portfolio_cumsum[i] = initial + portfolio_cumsum[i]
        initial = portfolio_cumsum[i]
    
    #1. shares table
    #2. costs table
    #3. cash table
    #4. portfolio without transaction costs table 
    #5. portfolio with transaction cost table
    #6. portfolio return P&L table
    #7. portfolio accumulative return table
    return(balance_share,
           balance_cost,
           balance_cash,
           portfolio,
           portfolio_cost,
           portfolio_return,
           portfolio_cumsum)


In [103]:
#calculate mean variance allocation portolio
(balance_share_meanv,
 balance_cost_meanv,
 balance_cash_meanv,
 portfolio_meanv,
 portfolio_cost_meanv,
 portfolio_return_meanv,
 portfolio_cumsum_meanv)=cal_portfolio(stocks_name,tradedate_full, weight_meanv, capital, transaction_percent)


Missing Dates: {'2022-06-01', '2020-09-01', '2019-03-01', '2024-03-01', '2020-06-01', '2020-12-01', '2021-06-01', '2023-03-01', '2023-06-01', '2022-12-01', '2021-09-01', '2023-09-01', '2021-03-01', '2018-03-01', '2022-03-01', '2022-09-01', '2018-06-01', '2023-12-01', '2021-12-01'}
balance_cost shape: (607, 0)
balance_price shape: (607, 0)


ValueError: 🚨 Error: `balance_cost` is empty. Check its computation earlier.

In [69]:
sum(portfolio_return_meanv)

NameError: name 'portfolio_return_meanv' is not defined

In [ ]:
#calculate minimum variance allocation portolio
(balance_share_minv,
 balance_cost_minv,
 balance_cash_minv,
 portfolio_minv,
 portfolio_cost_minv,
 portfolio_return_minv,
 portfolio_cumsum_minv)=cal_portfolio(stocks_name,tradedate_full, weight_minv, capital, transaction_percent)

In [ ]:
#calculate equally weighted allocation portolio
(balance_share_equal,
 balance_cost_equal,
 balance_cash_equal,
 portfolio_equal,
 portfolio_cost_equal,
 portfolio_return_equal,
 portfolio_cumsum_equal)=cal_portfolio(stocks_name,tradedate_full, weight_equal, capital, transaction_percent)

In [ ]:
sum(portfolio_return_equal)

# 3. Calculate SPX Portfolio Value, Return and Cumulative Return

In [ ]:
import datetime

dates = pd.date_range('1999-09-01', '2022-09-01')


In [ ]:
spx = pd.read_csv("SPX.csv")


In [ ]:
spx.date = pd.to_datetime(spx.date)

In [ ]:
spx = spx.set_index("date").reindex(dates, method='ffill')

In [ ]:
spx

In [ ]:
tradedate = tradedate[:-1]

In [ ]:

spx_daily = list(set(spx.index))
spx_daily.sort()
spx = spx['close']
spx.index = spx.index.astype(str)
spx = spx[tradedate]
spx.index = range(len(spx))
spx_return = (spx - spx.shift(1))/spx
spx_return[0] = 0

spx_return.index = tradedate
spx_return.index = range(len(spx))
spx_cumsum=spx_return.copy()

spx_value = spx_return.copy()
initial = 0
for i in range(len(spx_cumsum)):
    spx_cumsum[i] = initial + spx_cumsum[i]
    initial = spx_cumsum[i]

initial = 1000000
for i in range(len(spx_value)):
    spx_value[i] = initial * (1 + spx_value[i])
    initial = spx_value[i]

spx_cumsum.index = tradedate
spx_value.index = tradedate

In [ ]:
sum(spx_return)

# 4. Plot the Portfolio Performance

In [ ]:
time_ind.head()

## 4.1 Plot Portfolio Value with Transaction Cost

In [ ]:
time_ind = tradedate_full[:-1]

In [ ]:
trace0 = go.Scatter(x = time_ind, y = portfolio_cost_meanv, mode = 'lines', name = 'Mean-Variance')
trace1 = go.Scatter(x = time_ind, y = portfolio_cost_equal, mode = 'lines', name = 'Equally Weighted')
trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

data = [trace0, trace1, trace2, trace3]
layout = dict(title = "Portfolio Value Performance Quarterly with Transaction cost", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)

plotly.offline.iplot(fig)

#portfolio_tmp = portfolio_cost_value

## 4.2 Plot Portfolio Return

In [ ]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_return_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_return_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_return_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_return, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

## 4.3 Plot Portfolio Cumulative Return

In [ ]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_cumsum_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_cumsum_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_cumsum_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_cumsum, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Cumulative Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

# 5. Get all Quarterly Return and Calcualte Annulized return, standard deviation and Sharpe ratio

In [ ]:
quarter_return = pd.DataFrame(index = range(len(portfolio_return_equal)), columns = ['Mean-Var', 'Equally', 'Min-Var'])
quarter_return['SPX'] = spx_return
quarter_return['Mean-Var'] = portfolio_return_meanv
quarter_return['Equally'] = portfolio_return_equal
quarter_return['Min-Var'] = portfolio_return_minv

#writer_return = pd.ExcelWriter('quarter_return_user8.xlsx')
quarter_return.index=tradedate_full
#quarter_return.to_excel(writer_return,'Sheet1')
#writer_return.save()

In [ ]:
quarter_return.head()

In [ ]:
annulizedR_spx=quarter_return['SPX'].mean()*4
annulizedR_meanv=quarter_return['Mean-Var'].mean()*4
annulizedR_equal=quarter_return['Equally'].mean()*4
annulizedR_minv=quarter_return['Min-Var'].mean()*4
annulizedS_spx=quarter_return['SPX'].std()*2
annulizedS_meanv=quarter_return['Mean-Var'].std()*2
annulizedS_equal=quarter_return['Equally'].std()*2
annulizedS_minv=quarter_return['Min-Var'].std()*2

In [ ]:
print("SPX",annulizedR_spx,annulizedS_spx)
print("Mean-Var",annulizedR_meanv,annulizedS_meanv)
print("Equally",annulizedR_equal,annulizedS_equal)
print("Min-Var",annulizedR_minv,annulizedS_minv)


In [ ]:
risk_free=0.015

In [ ]:
sharpe_spx=(annulizedR_spx-risk_free)/annulizedS_spx
sharpe_meanv=(annulizedR_meanv-risk_free)/annulizedS_meanv
sharpe_equal=(annulizedR_equal-risk_free)/annulizedS_equal
sharpe_minv=(annulizedR_minv-risk_free)/annulizedS_minv

In [ ]:
print("SPX Sharpe",sharpe_spx)
print("Mean-Var Sharpe",sharpe_meanv)
print("Equally Sharpe",sharpe_equal)
print("Min-Var Sharpe",sharpe_minv)


In [ ]:
insample_date=tradedate

In [ ]:
insample_date

In [ ]:
quarter_return['SPX'][insample_date]


In [ ]:
annulizedR_spx_insample=quarter_return['SPX'][insample_date].mean()*4
annulizedR_meanv_insample=quarter_return['Mean-Var'][insample_date].mean()*4
annulizedR_equal_insample=quarter_return['Equally'][insample_date].mean()*4
annulizedR_minv_insample=quarter_return['Min-Var'][insample_date].mean()*4
annulizedS_spx_insample=quarter_return['SPX'][insample_date].std()*2
annulizedS_meanv_insample=quarter_return['Mean-Var'][insample_date].std()*2
annulizedS_equal_insample=quarter_return['Equally'][insample_date].std()*2
annulizedS_minv_insample=quarter_return['Min-Var'][insample_date].std()*2
print("SPX",annulizedR_spx_insample,annulizedS_spx_insample)
print("Mean-Var",annulizedR_meanv_insample,annulizedS_meanv_insample)
print("Equally",annulizedR_equal_insample,annulizedS_equal_insample)
print("Min-Var",annulizedR_minv_insample,annulizedS_minv_insample)
sharpe_spx_insample=(annulizedR_spx_insample-risk_free)/annulizedS_spx_insample
sharpe_meanv_insample=(annulizedR_meanv_insample-risk_free)/annulizedS_meanv_insample
sharpe_equal_insample=(annulizedR_equal_insample-risk_free)/annulizedS_equal_insample
sharpe_minv_insample=(annulizedR_minv_insample-risk_free)/annulizedS_minv_insample
print("SPX Sharpe",sharpe_spx_insample)
print("Mean-Var Sharpe",sharpe_meanv_insample)
print("Equally Sharpe",sharpe_equal_insample)
print("Min-Var Sharpe",sharpe_minv_insample)